<a href="https://colab.research.google.com/github/Vikas-KM/tensorflow-learning/blob/master/cnn-cifar20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [34]:
# load the dataset
(X_train, y_train),(X_test, y_test) = cifar10.load_data()

In [35]:
# convert to float32 and normalize the data
X_train = X_train.astype('float32')/255.0 
X_test = X_test.astype('float32')/255.0

In [36]:
# Sequential
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)),
     layers.Conv2D(32, 3, padding='valid', activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64,3, activation='relu'),
     layers.MaxPooling2D(),
     layers.Conv2D(128,3, activation='relu'),
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(10),
    ]
)

In [37]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                1

In [38]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics = ['accuracy']
)

In [39]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
782/782 - 2s - loss: 1.6749 - accuracy: 0.3928
Epoch 2/10
782/782 - 2s - loss: 1.3509 - accuracy: 0.5205
Epoch 3/10
782/782 - 2s - loss: 1.2291 - accuracy: 0.5665
Epoch 4/10
782/782 - 2s - loss: 1.1373 - accuracy: 0.6000
Epoch 5/10
782/782 - 2s - loss: 1.0568 - accuracy: 0.6309
Epoch 6/10
782/782 - 2s - loss: 0.9856 - accuracy: 0.6570
Epoch 7/10
782/782 - 2s - loss: 0.9335 - accuracy: 0.6752
Epoch 8/10
782/782 - 2s - loss: 0.8863 - accuracy: 0.6932
Epoch 9/10
782/782 - 2s - loss: 0.8416 - accuracy: 0.7110
Epoch 10/10
782/782 - 2s - loss: 0.7996 - accuracy: 0.7231


In [40]:
model.evaluate(X_test, y_test, batch_size=64, verbose=2)

157/157 - 0s - loss: 0.8906 - accuracy: 0.6913


[0.8905830979347229, 0.6912999749183655]

### Functional with regularization and dropout added

In [27]:
# Functional
def my_model():
  inputs = keras.Input(shape=(32,32,3))
  x = layers.Conv2D(32,3, padding='same', kernel_regularizer=regularizers.l2(0.01))(inputs)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(64,3, padding='same', kernel_regularizer = regularizers.l2(0.01))(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(128,3, padding='same', kernel_regularizer = regularizers.l2(0.01))(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.01))(x)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model


In [28]:
model = my_model()

In [29]:
print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
tf.nn.relu_9 (TFOpLambda)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_10 (Batc (None, 16, 16, 64)        256 

In [30]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=3e-4),
    metrics=['accuracy'],
)

In [31]:
model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=2)

Epoch 1/50
782/782 - 3s - loss: 3.0873 - accuracy: 0.3459
Epoch 2/50
782/782 - 3s - loss: 1.9522 - accuracy: 0.4683
Epoch 3/50
782/782 - 3s - loss: 1.6423 - accuracy: 0.5228
Epoch 4/50
782/782 - 3s - loss: 1.5097 - accuracy: 0.5500
Epoch 5/50
782/782 - 3s - loss: 1.4366 - accuracy: 0.5741
Epoch 6/50
782/782 - 3s - loss: 1.3873 - accuracy: 0.5870
Epoch 7/50
782/782 - 3s - loss: 1.3511 - accuracy: 0.6017
Epoch 8/50
782/782 - 3s - loss: 1.3223 - accuracy: 0.6094
Epoch 9/50
782/782 - 3s - loss: 1.3090 - accuracy: 0.6163
Epoch 10/50
782/782 - 3s - loss: 1.2795 - accuracy: 0.6251
Epoch 11/50
782/782 - 3s - loss: 1.2658 - accuracy: 0.6304
Epoch 12/50
782/782 - 3s - loss: 1.2597 - accuracy: 0.6365
Epoch 13/50
782/782 - 3s - loss: 1.2437 - accuracy: 0.6400
Epoch 14/50
782/782 - 3s - loss: 1.2296 - accuracy: 0.6474
Epoch 15/50
782/782 - 3s - loss: 1.2183 - accuracy: 0.6520
Epoch 16/50
782/782 - 3s - loss: 1.2103 - accuracy: 0.6565
Epoch 17/50
782/782 - 3s - loss: 1.1966 - accuracy: 0.6616
Epoch 

In [32]:
model.evaluate(X_test, y_test, batch_size=64, verbose=2)

157/157 - 1s - loss: 1.4140 - accuracy: 0.6276


[1.4140430688858032, 0.6276000142097473]